# **Tuition Analyzer**

### Importing the required packages

In [1]:
import pandas as pd
import numpy as np

In [2]:
def tuition_parser(sheet_dict):
    """
    Function for parsing an excel doc with multiple tabs from AAMC.
    sheet_dict is the sheets dictionary.
    Renames columns without the \n character and drops the footer with the @ AAMC claim.
    Returns a summary statistic df and a concatenated df by year/cycle
    """
    all_sheets = []
    for name, sheet in sheet_dict.items():
        # Naming a column in each df by it's excel tab name and replacing any \n characters
        sheet["Cycle"] = name
        sheet.columns = [col.replace("\n", " ") for col in sheet.columns]
        
        # Dropping all NA values from the first column, and dropping the rows after the row that contains the @ AAMC claim
        sheet = sheet[sheet.iloc[:,0].notna()]
        cut_index = int(sheet[sheet.iloc[:,0].str.contains("©")].index.values)
        index_exclude = sheet.iloc[cut_index: , :].index
        sheet = sheet.drop(index_exclude, axis=0)
        
        # Appending all sheets in a list
        all_sheets.append(sheet)
    
    # The summary statistics is the first index (first excel tab)
    summary = all_sheets[0]
    
    # Dropping the summary statistics off the list and concatening all remaining cycle sheets
    del all_sheets[0]
    tuition = pd.concat(all_sheets).reset_index(drop=True)
    
    # Returning the summary statustics and concatenated tuition data by cycle/year
    return(summary, tuition)

### Importing all excel docs with all their tabs into 3 sheet dictionaries.  Cleaning

In [3]:
sheets_dict1 = pd.read_excel("AAMC Data/Tuition and Student Fees Report, 1995-1996 through 2005-2006.xlsx", sheet_name=None, skiprows=6)
sheets_dict2 = pd.read_excel("AAMC Data/Tuition and Student Fees Report, 2006-2007 through 2012-2013.xlsx", sheet_name=None, skiprows=6)
sheets_dict3 = pd.read_excel("AAMC Data/Tuition and Student Fees Report, 2013-2014 through 2021-2022.xlsx", sheet_name=None, skiprows=6)

In [4]:
# Running the tuittion parser function to merge/clean dfs
summary1, tuition1 = tuition_parser(sheet_dict=sheets_dict1)
summary2, tuition2 = tuition_parser(sheet_dict=sheets_dict2)
summary3, tuition3 = tuition_parser(sheet_dict=sheets_dict3)

# Concatenating the 3 dfs from the 3 excel files together
summary = pd.concat([summary1, summary2, summary3]).reset_index(drop=True)
tuition = pd.concat([tuition1, tuition2, tuition3]).reset_index(drop=True)

# Creating a "Year" integer column and dropping the cycle column from the summary df
summary["Year"] = [int(y[0:4]) for y in summary["Academic Year"]]
summary = summary.drop("Cycle", axis=1)
tuition["Year"] = [int(y[0:4]) for y in tuition["Cycle"]]

# Converting all money amounts to floats in the tuition["Reported Cost"] column
new_money = []
for money in tuition["Reported Cost"]:
    if isinstance(money, str):
        money = money.replace("$", "")
        money = money.replace(",", "")
        try:
            money = float(money)
            new_money.append(money)
        except:
            money = np.nan
            new_money.append(money)
    else:
        new_money.append(money)
tuition["Reported Cost"] = new_money

# Sorting and reordering columns to make things easier to view
summary = summary.sort_values(["Year", "Cost Type", "Residence Status", "Ownership Type"]).reset_index(drop=True)
summary_cols = ['Academic Year', 'Year', 'Cost Type', 'Ownership Type', 'Residence Status', 'Minimum Cost', 'Median Cost', 'Maximum Cost', 'Average Cost', 'Average Cost  Percent Change  from Prior Year', 'Participating  Medical Schools  by Year and  Ownership']
summary = summary[summary_cols]
tuition = tuition.sort_values(["Year", "Medical School Name", "Cost Type", "Residence Status"]).reset_index(drop=True)
tuition_cols = ['Cycle', 'Year', 'Medical School Name', 'Short Name', 'Cost Type', 'Ownership Type', 'Residence Status', 'Health Insurance Required ', 'Health Insurance Waived', 'Reported Cost']
tuition = tuition[tuition_cols]

# Viewing both the summary and tuition dfs 
display(summary)
display(tuition)

,Academic Year,Year,Cost Type,Ownership Type,Residence Status,Minimum Cost,Median Cost,Maximum Cost,Average Cost,Average Cost Percent Change from Prior Year,Participating Medical Schools by Year and Ownership
0,1995-1996,1995,Fees,Private,Nonresident,0.0,825.0,3720.0,888.0,NaN,50.0
1,1995-1996,1995,Fees,Public,Nonresident,0.0,589.0,15164.0,1541.0,NaN,74.0
2,1995-1996,1995,Fees,Private,Resident,0.0,760.0,3720.0,843.0,NaN,50.0
3,1995-1996,1995,Fees,Public,Resident,0.0,581.0,8619.0,1285.0,NaN,74.0
4,1995-1996,1995,Tuition,Private,Nonresident,14400.0,23200.0,30300.0,23171.0,NaN,50.0
...,...,...,...,...,...,...,...,...,...,...,...
399,2021-2022,2021,Tuition,Public,Resident,0.0,34577.0,57936.0,33489.0,0.029,88.0
400,2021-2022,2021,"Tuition, Fees, and Health Insurance",Private,Nonresident,0.0,66635.0,73935.0,63630.0,0.080,60.0
401,2021-2022,2021,"Tuition, Fees, and Health Insurance",Public,Nonresident,0.0,65753.0,91599.0,62960.0,0.070,88.0
402,2021-2022,2021,"Tuition, Fees, and Health Insurance",Private,Resident,0.0,65580.0,73935.0,62179.0,0.030,60.0


,Cycle,Year,Medical School Name,Short Name,Cost Type,Ownership Type,Residence Status,Health Insurance Required,Health Insurance Waived,Reported Cost
0,1995-96,1995,Albany Medical College,Albany,Fees,Private,Nonresident,NaN,NaN,0.0
1,1995-96,1995,Albany Medical College,Albany,Fees,Private,Resident,NaN,NaN,0.0
2,1995-96,1995,Albany Medical College,Albany,Tuition,Private,Nonresident,NaN,NaN,25346.0
3,1995-96,1995,Albany Medical College,Albany,Tuition,Private,Resident,NaN,NaN,24074.0
4,1995-96,1995,Albany Medical College,Albany,Tuition and Fees,Private,Nonresident,NaN,NaN,25346.0
...,...,...,...,...,...,...,...,...,...,...
27035,2021-22,2021,Yale School of Medicine,Yale,Health Insurance,Private,Resident,Yes,Yes,2758.0
27036,2021-22,2021,Yale School of Medicine,Yale,Tuition,Private,Nonresident,Yes,Yes,66160.0
27037,2021-22,2021,Yale School of Medicine,Yale,Tuition,Private,Resident,Yes,Yes,66160.0
27038,2021-22,2021,Yale School of Medicine,Yale,"Tuition, Fees, and Health Insurance",Private,Nonresident,Yes,Yes,69981.0


### Exporting to Excel

In [5]:
#tuition.to_excel("Tuitions Merged.xlsx", index=False)
#summary.to_excel("Summary Statistics Merged.xlsx", index=False)